In [2]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
from skimage.feature._canny import canny
import sys
%matplotlib inline

In [3]:
# filter a copy of an image for color
def process_image_color(img, rgb_threshold):
    color_select = np.copy(img)

    # ### Mask pixels below the threshold
    color_thresholds = (color_select[:,:,0] < rgb_threshold[0]) | \
        (color_select[:,:,1] < rgb_threshold[1]) | \
        (color_select[:,:,2] < rgb_threshold[2])
    return (color_select, color_thresholds)

In [4]:
# Define trianglular region
def build_triangle(img, apex, right, left):
    # ### Get Image size 
    ysize = img.shape[0]
    xsize = img.shape[1]
    # ### Define the vertices of a quadrillateral mask.
    # Keep in mind the origin (x=0, y=0) is in the upper left
    apex = [apex[0]*xsize, apex[1]*ysize]
    right_bottom = [right*xsize, ysize-1]
    left_bottom = [left*xsize, ysize-1]
    
    # Fit lines (y=Ax+B) to identify the  3 sided region of interest
    # np.polyfit() returns the coefficients [A, B] of the fit
    fit_left = np.polyfit((left_bottom[0], apex[0]), (left_bottom[1], apex[1]), 1)
    fit_right = np.polyfit((right_bottom[0], apex[0]), (right_bottom[1], apex[1]), 1)
    fit_bottom = np.polyfit((left_bottom[0], right_bottom[0]), (left_bottom[1], right_bottom[1]), 1)
    
    # Find the region inside the lines
    XX, YY = np.meshgrid(np.arange(0, xsize), np.arange(0, ysize))
    region_thresholds = (YY > (XX*fit_left[0] + fit_left[1])) & \
                        (YY > (XX*fit_right[0] + fit_right[1])) & \
                        (YY < (XX*fit_bottom[0] + fit_bottom[1]))
    return region_thresholds

In [5]:
# mask image based on color and region limits
def mask_image(img, color_thresholds, region_thresholds):
    # Mask color and region selection
    img[color_thresholds | ~region_thresholds] = [0, 0, 0]
    return img

In [6]:
def process_canny(color_select, kernel_size, canny_thresholds):
    gray = cv2.cvtColor(color_select, cv2.COLOR_RGB2GRAY)
    blur_gray = cv2.GaussianBlur(gray,(kernel_size, kernel_size),0)
    # Execute Canny
    # Define our parameters for Canny and run it
    edges = cv2.Canny(blur_gray, canny_thresholds[0], canny_thresholds[1])
    # Display the image
    return edges

In [7]:
def hough_transform(hough_params, image):
    
    # Run Hough on edge detected image
    # Output "lines" is an array containing endpoints of detected line segments
    lines = cv2.HoughLinesP(image, hough_params['rho'], 
                            hough_params['theta'], 
                            hough_params['threshold'], 
                            np.array([]),
                            hough_params['min_line_length'], 
                            hough_params['max_line_gap'])
    return lines

In [8]:
def add_lanelines(img, lines):
    # Iterate over the output "lines" and draw lines on a blank image
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img,(x1,y1),(x2,y2),(0,0,255),2)
    return img

In [9]:
def get_lanelines(color_select, params, color_thresholds, debug=False):
    region_thresholds = build_triangle(color_select, params['apex'],
                                       params['right'], params['left'])
    
    color_select = mask_image(color_select, color_thresholds, region_thresholds)
    if debug:
        display_image("masked image",color_select)
    edges = process_canny(color_select, params['kernel_size'], params['canny_thresholds'])
    if debug:
        display_image("canny output", edges)
    lines = hough_transform(params['hough_params'], edges)
    return lines

In [10]:
def define_params():
    params = {}
    # ### Define color selection criteria
    red_threshold = 100
    green_threshold = 100
    blue_threshold = 0
    params['rgb'] = [red_threshold, green_threshold, blue_threshold]
    
    # Define a kernel size for Gaussian smoothing / blurring
    params['kernel_size'] = 5 # Must be an odd number (3, 5, 7...)
    # define canny low and high thresholds
    params['canny_thresholds'] = (100, 200)
    # Define the Hough transform parameters
    hough_params = {}
    hough_params['rho'] = 2 # distance resolution in pixels of the Hough grid
    hough_params['theta'] = np.pi/180 # angular resolution in radians of the Hough grid
    hough_params['threshold'] = 25     # minimum number of votes (intersections in Hough grid cell)
    hough_params['min_line_length'] = 100 #minimum number of pixels making up a line
    hough_params['max_line_gap'] = 40    # maximum gap in pixels between connectable line segments
    params['hough_params'] = hough_params
    params['apex'] = (0.48, 0.58)
    params['right'] = 0.93
    params['left'] = 0.15
    return params

In [11]:
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image with lines are drawn on lanes)
    params = define_params()
    (color_select, color_thresholds) = process_image_color(image, params['rgb'])
    lines = get_lanelines(color_select, params, color_thresholds)
    result = image
    if lines is not None:
        result = add_lanelines(image, lines)
    return result

In [12]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [13]:
white_output = 'white.mp4'
clip1 = VideoFileClip("solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video white.mp4
[MoviePy] Writing video white.mp4


100%|███████████████████████████████████████▊| 221/222 [00:09<00:00, 23.84it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: white.mp4 

Wall time: 9.67 s


In [14]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

In [17]:
yellow_output = 'yellow.mp4'
clip2 = VideoFileClip('solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

[MoviePy] >>>> Building video yellow.mp4
[MoviePy] Writing video yellow.mp4


100%|███████████████████████████████████████▉| 681/682 [00:26<00:00, 23.57it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: yellow.mp4 

Wall time: 26.9 s


In [19]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

In [20]:
challenge_output = 'extra.mp4'
clip2 = VideoFileClip('challenge.mp4')
challenge_clip = clip2.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

[MoviePy] >>>> Building video extra.mp4
[MoviePy] Writing video extra.mp4


100%|████████████████████████████████████████| 251/251 [00:17<00:00, 14.49it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: extra.mp4 

Wall time: 18.1 s


In [21]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))

## Reflections

The pipeline is as follows. 
First, one puts a color filter through to accentute the lighter colors. 

Second a visual triangle is setup to block out anything not directly in front of the vehicle and out to about half way up the picture, where perspective would have the lane lines merge. It is assumed that this triangle would be fixed for a given fixed camera on the vehicle. 

At that point, one can convert the picture to greyscale and perform canny and hough transforms. 
The result should be a pair of lines that can be superimposed on the original color frame.

The first videos have a clear shot at the road while the challenge does not. Short of adjusting the camera angle, ignoring the bottom 50 or 60 pixels. 

Also, it appears that the yellow color is darker and we're not picking it up in the filters. By changing the rgb threshold from (120, 120, 120) to (100,100,0) I was able to pickup more of the lane lines